#Initial Setup

In [ ]:
%%capture
!pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

In [ ]:
%%capture
!pip install transformers
!pip install SentencePiece

In [ ]:
# Import libraries
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import torch
import transformers

In [ ]:
# For reproducibility
SEED = 42

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [ ]:
#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

# Load Data



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### TO CHANGE ######
DIR = f"/content/gdrive/MyDrive/Path/to/Dementiabank/folder"

Mounted at /content/gdrive


In [ ]:
# Load the ids of AD subjects into a list
with open(f'{DIR}/audio_filenames_dementia.txt', "r") as clf:
    lines = clf.readlines()
ids_ad = [re.sub('\n', '', line) for line in lines]

# Load the ids of Control subjects into a list
with open(f'{DIR}/audio_filenames_control.txt', "r") as clf:
    lines = clf.readlines()
ids_hc = [re.sub('\n', '', line) for line in lines]

path_ad = 'path_to_ad_data'
path_hc = 'path_to_hc_data'
test_path = 'path_to_test_data'
test_id_path = 'path_to_a_txt_file_with_test_sample_labels'

data_ad, labels_ad, aug_dataset_ad = data_to_str(ids_ad, path_ad, AD_flag=1,
                                                 augment=False) 
data_hc, labels_hc, aug_dataset_hc = data_to_str(ids_hc, path_hc, AD_flag=0,
                                                 augment=False)
dataset, labels, aug_dataset = [], [], []

dataset.extend(data_ad)
dataset.extend(data_hc)


# Paraphrase

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences,num_beams):
  tokenizer = PegasusTokenizer.from_pretrained(model_name)
  batch = tokenizer(input_text,truncation=True,padding='longest',max_length=30, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=0.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
def paraphrase(text):
  text = text.replace('[SEP]','')
  txt = text.split('.')
  txt = [x for x in txt if len(x.strip())>0]
  num_beams = 10
  num_return_sequences = 1
  print(len(txt))

  start = 10
  end = min(20,len(txt))
  step = 10
  res = get_response(txt[:10],num_return_sequences,num_beams)
  
  while end!=len(txt):
    res.extend(get_response(txt[start:end],num_return_sequences,num_beams))
    start = end 
    end = min(start+step,len(txt))

  res_sent = ' '.join(res)

  return res_sent


In [ ]:
dataset['paraphrase'] = np.vectorize(paraphrase)(dataset)

# Inspect & Save

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv(f'{DIR}/paraphrase_aug_dataset.csv',index=False)